In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [4]:
from PIL import Image
from pathlib import Path
from torchvision import transforms
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageFolder

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()

In [6]:
test_data_path  = Path('/content/drive/MyDrive/Dataset/Testing')
train_data_path = Path('/content/drive/MyDrive/Dataset/Training')
image_path_list = list(Path('/content/drive/MyDrive/Dataset').glob("*/*/*.jpg"))

In [7]:
image_transform = transforms.Compose([
  transforms.Resize((224, 224)),
  transforms.ToTensor(),
  transforms.Normalize(mean = [0.1855, 0.1855, 0.1855], std = [0.2003, 0.2003, 0.2004])
])

train_data = ImageFolder(train_data_path, transform = image_transform)
train_loader = DataLoader(train_data, batch_size = 8, shuffle = True)
test_data = ImageFolder(test_data_path, transform = image_transform)
test_loader = DataLoader(test_data, batch_size = 8, shuffle = False)

In [14]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    self.features = nn.Sequential(
      nn.Conv2d(3, 16, kernel_size = 3, stride = 1, padding = 1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size = 2, stride = 2, return_indices = True),

      nn.Conv2d(16, 16, kernel_size = 3, stride = 1, padding = 1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size = 2, stride = 2, return_indices = True),

      nn.Conv2d(16, 16, kernel_size = 3, stride = 1, padding = 1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size = 2, stride = 2, return_indices = True)
    )

    self.switch_indices = [0] * len(self.features)

    self.classifier = nn.Sequential(
        nn.Linear(16 * 28 * 28, 64),
        nn.ReLU(),
        nn.Linear(64, 4) # Image classes = 4
    )

  def forward(self, batch: torch.Tensor):
    for i, layer in enumerate(self.features):
      if isinstance(layer, nn.MaxPool2d):
        batch, indices = layer(batch)
        self.switch_indices[i] = indices
      else:
        batch = layer(batch)

    batch = batch.view(batch.size(0), -1)
    batch_probabilities = self.classifier(batch)

    return batch_probabilities

In [15]:
def train_model(model, num_epochs, train_loader, optimizer, loss_fn):
  for epoch in range(num_epochs):
    model.train()

    correct_predictions = 0
    total_samples = 0

    for _, (images, labels) in enumerate(train_loader):
      images, labels = images.to(device), labels.to(device)

      optimizer.zero_grad()
      batch_probabilities = model(images)
      loss = loss_fn(batch_probabilities, labels)
      loss.backward()

      optimizer.step()

      _, predicted_labels = torch.max(batch_probabilities, 1)
      correct_predictions += (predicted_labels == labels).sum().item()
      total_samples += predicted_labels.size(0)

    train_accuracy = correct_predictions / total_samples
    print(f'Epoch: {epoch} Train accuracy: {train_accuracy}')

In [16]:
@torch.no_grad()
def test_model(model, test_loader):
  model.eval()

  correct_predictions = 0
  total_samples = 0

  for inputs, labels in test_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)
    total_samples += labels.size(0)
    correct_predictions += (predicted == labels).sum().item()

  accuracy = correct_predictions / total_samples
  print(f"Accuracy: {accuracy:.2%}")

In [18]:
model = CNN().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [19]:
train_model(model, 10, train_loader, optimizer, loss_fn)

Epoch: 0 Train accuracy: 0.7930672268907563
Epoch: 1 Train accuracy: 0.9108893557422969
Epoch: 2 Train accuracy: 0.9483543417366946
Epoch: 3 Train accuracy: 0.9698879551820728
Epoch: 4 Train accuracy: 0.9772408963585434
Epoch: 5 Train accuracy: 0.9879201680672269
Epoch: 6 Train accuracy: 0.9858193277310925
Epoch: 7 Train accuracy: 0.9956232492997199
Epoch: 8 Train accuracy: 0.990546218487395
Epoch: 9 Train accuracy: 0.993172268907563


In [20]:
test_model(model, test_loader)

Accuracy: 94.69%


In [21]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Models/2411_cnn_model.pth')